# Reformatting Louisiana Injection data

In [1]:
import pandas as pd


In [14]:
wells = pd.read_csv(r"C:\MyDocs\OpenFF\data\non-FF\louisiana\LA_well_info.csv",encoding='ISO-8859-1',
                   low_memory=False)
print(wells.columns)
# prune it
wells = wells[['Well Serial Num','API Num','Latitude','Longitude']]
wells.head(5)

Index(['Operator Name', 'Operator ID', 'Field ID', 'Field Name',
       'Well Serial Num', 'Well Name', 'Well Num', 'Lease Num',
       'Well Status Code', 'Well Status Code Description', 'Classification',
       'Well Class Type Code', 'Well Class Type Code Description', 'API Num',
       'Effective Date', 'Permit Date', 'Spud Date', 'Well Status Date',
       'Section', 'Township', 'Range', 'Meridian', 'Parish Code',
       'Parish Name', 'District Code', 'District Name', 'Ground Elevation',
       'Latitude', 'Longitude', 'Product Type Code',
       'Product Type Code Description', 'USDW Value', 'Area USDW Value',
       'Source Area USDW Value'],
      dtype='object')


,Well Serial Num,API Num,Latitude,Longitude
0,974063,17073880810000,32.2775570622541,-92.053505691534
1,229443,17021214430000,32.1695434275678,-92.0912573202293
2,229458,17021214440000,32.1663092509176,-92.0857130360819
3,229459,17021214450000,32.1636914552075,-92.0922527826289
4,232843,17021214610000,32.1586431677212,-92.0867492679023


It looks like Reporting dates (start and stop) are within the same year but are not necessarily 365 days.  Take a difference between stop and start to get #days.

In [29]:
inj = pd.read_csv(r"C:\MyDocs\OpenFF\data\non-FF\louisiana\saltwater_disposal_report.csv",
                  encoding='ISO-8859-1',
                  low_memory=False)
print(inj.columns)
inj['start_date'] = pd.to_datetime(inj['Report Begin Date'])
inj['end_date'] = pd.to_datetime(inj['Report End Date'])
inj['num_days'] = (inj.end_date - inj.start_date).dt.days
inj['year'] = inj.start_date.dt.year
# prune
inj = inj[['Well Serial Num','year','num_days','Total Volume Injected (Bbls)',
       'Total Produced Fluids']]

inj.columns = ['Well Serial Num','year','num_days','vol_inj','pfluids']

# because raw data has dashes, it is not converted to numeric. We have to do that explicitly.
inj.vol_inj = pd.to_numeric(inj.vol_inj,errors='coerce')
inj.pfluids = pd.to_numeric(inj.pfluids,errors='coerce')



Index(['&nbsp;', 'Report Begin Date', 'Report End Date', 'Report Completed By',
       'Completor Phone', 'Operator ID', 'Operator Name', 'Well Serial Num',
       'UIC10 Received Date', 'Injection Through Method', 'Packer Depth',
       'Avg Injection Pressure', 'Max Injection Pressure',
       'Min Annulus Pressure', 'Max Annulus Pressure', 'Avg Injection Rate',
       'Max Injection Rate', 'Total Volume Injected (Bbls)',
       'Total Produced Fluids', 'MASIP', 'Over MASIP', 'Comments',
       'Well Status Code', 'Classification', 'Well Class Type Code', 'Field',
       'Parish', 'Interval From', 'Interval To', 'Community Curr Indicator'],
      dtype='object')


In [30]:
# summarize by year and then pivot
gb = inj.groupby(['Well Serial Num','year'],as_index=False)[['vol_inj','pfluids']].sum()
gb.tail(10)

,Well Serial Num,year,vol_inj,pfluids
28975,990623,2012,73200.0,73200.0
28976,990623,2013,456250.0,456250.0
28977,990623,2014,493000.0,493000.0
28978,990623,2015,511000.0,511000.0
28979,990623,2016,511000.0,511000.0
28980,990623,2017,385000.0,385000.0
28981,990623,2018,504000.0,504000.0
28982,990623,2019,425600.0,425600.0
28983,990623,2020,469000.0,469000.0
28984,990623,2021,425600.0,425600.0


In [31]:
colnames = ['vol_inj','pfluids']
concat_list = []
for col in colnames:
    piv = gb.pivot(index='Well Serial Num',columns='year',values=col).fillna(0)
    names = piv.columns.tolist()
    ncols = []
    for name in names:
        ncols.append(col+'_'+str(name))
    piv.columns = ncols
    piv[f'{col}_Total'] = piv.sum(axis=1)
    concat_list.append(piv)

whole = pd.concat(concat_list,axis=1)
whole.head()

,vol_inj_2011,vol_inj_2012,vol_inj_2013,vol_inj_2014,vol_inj_2015,vol_inj_2016,vol_inj_2017,vol_inj_2018,vol_inj_2019,vol_inj_2020,...,pfluids_2013,pfluids_2014,pfluids_2015,pfluids_2016,pfluids_2017,pfluids_2018,pfluids_2019,pfluids_2020,pfluids_2021,pfluids_Total
Well Serial Num,,,,,,,,,,,,,,,,,,,,,
1105,0.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,...,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,1800000.0
1203,0.0,0.0,900866.0,936389.0,550140.0,1109762.0,977812.0,1034127.0,745321.0,593334.0,...,900866.0,936389.0,550140.0,1109762.0,977812.0,1034127.0,745321.0,593334.0,712689.0,7560440.0
1266,0.0,0.0,1426537.0,1511974.0,1468707.0,1707667.0,1507618.0,948123.0,909799.0,980292.0,...,1426537.0,1511974.0,1468707.0,1707667.0,1507618.0,948123.0,909799.0,980292.0,963570.0,11424287.0
1685,0.0,0.0,649197.0,764850.0,1150632.0,338416.0,279685.0,378876.0,215026.0,208107.0,...,649197.0,764850.0,1150632.0,338416.0,279685.0,378876.0,215026.0,208107.0,166447.0,4151236.0
1713,0.0,0.0,1409842.0,1412592.0,1401867.0,1516594.0,1559731.0,1358972.0,1372499.0,1219948.0,...,1409842.0,1412592.0,1401867.0,1516594.0,1559731.0,1358972.0,1372499.0,1219948.0,930830.0,12182875.0


In [32]:
out = pd.merge(wells[['Well Serial Num','API Num','Longitude', 'Latitude']],whole,
               on='Well Serial Num',how='right',validate='1:1')

out.to_csv('LA_injection_summary.csv',index=False)